<a href="https://colab.research.google.com/github/LosanChar/Procesamiento-de-Imagenes/blob/main/PDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Medición de color a traves de dispositivos Móviles usando Colab**




# *Abstract*

1.- Estado del problema
El problema de saber que color es un color especifico, se vuelve subjetivo cuando lo trasponemos a la opinion de otro ser humano, pues bajo las experiencias y vivencias de cada persona, un color puede ser interpretado como otro. Es decir, para alguien el color azul cielo puede ser un azul mas claro o mas oscuro. En la actualidad donde requerimos de precisión en cuanto a lo que se estudia, se requiere muchas veces analizar de que tonalidades del entorno estamos ocupando. Por ello, en la edad media se llevo a cabo un estudio por el color de forma muy distinta. De tal forma que asentó las bases para la elaboración de un colorímetro.

# Introduccion

En la vida diaria, estamos rodeados por la naturaleza y por objetos que tienen colores bien conocidos. Sin embargo, existen diferencias de observación que pueden definir el mismo color de manera diferente, ya que la percepción visual
de cada persona varía de acuerdo a su conocimiento, criterio, sensibilidad o experiencia, por lo que para una persona el color del cielo puede ser “azul cielo”; para otra es “azul claro”; aquí ya se está agregando un atributo más al color, por lo que la expresión verbal de los colores es subjetiva y compleja, ya que también hay factores que intervienen en la apariencia de éstos, como son: la fuente de luz, el observador, el tamaño, el fondo, el ángulo de observación, entre otros. 

Un gran adelanto fue el colorímetro de Duboscq, quien desarrolló un instrumento para medir, variando la altura de las muestras, su relación entre la patrón y la desconocida. La luz necesaria era proporcionada por el sol mediante un espejo, como los primeros microscopios. Varios instrumentos se desarrollaron según ese principio, desde simples comparadores ópticos hasta complejos instrumentos de medición.

Con el advenimiento de la electrónica, se fue mejorando la implementación instrumental y se añadieron fotocélulas para reemplazar al ojo humano. De allí el agregado de "foto" y el término devino en fotocolorímetro. Eso facilitó los análisis químicos y dio nacimiento a los actuales instrumentos, tanto los manuales como los grandes autoanalizadores químicos que con complejos mecanismos electrónicos y mecánicos realizan toda la tarea del químico operador.

La contribuciones de este documento son los resultados, que nos brinda la precisión de la computación, los cuales obtenemos a traves del uso de herramientas de fácil acceso, tales como Colab de Google, lenguaje Python y Javascript. De tal forma que los resultados obtenidos a traves de las pruebas que se detallan en este documento, se pueden llegar a obtener dichos resultados en cualquier lugar.



![Figura 1](https://drive.google.com/uc?id=1poe8tPaJhWqQEeOqEB_DewCBMrro5qe8)

*Figura 1. Coordenadas cromáticas de todos los colores medidos para cuatro escenas de prueba. Los círculos corresponden a las mediciones del fotopectrómetro y los cruces a los resultados de calibración para Canon 550D. La calibración para otras cámaras produjo resultados similares. La línea continua indica la gama de colores sRGB y la línea discontinua la gama Adobe RGB.*



#Diseño

La sección de diseño presenta el diseño de su sistema y puede incluir un párrafo sobre detalles de implementación.

Usando la herramienta de entorno de programación de Colab se realizaron cuatro importantes bloques. Donde el primero será la declaración de las bibliotecas para ocupar; la segunda ocupará las líneas de código en javascript que nos ayudará a que el entorno active la cámara del dispositivo y capture una imágen, la cual será la que ocuparemos para las mediciones de color;  

Aqui el primer bloque, como ya se menciono, declara nuestras bibliotecas a ocupar. 
Una de las más importantes es la importación de Javascript, esta biblioteca que nos permite ejecutar código de javascript dentro de un entorno de trabajo de python. 
De igual forma en cuanto a importancia, colour-science es un paquete de Python de código abierto que proporciona una gran cantidad de algoritmos y conjuntos de datos para la ciencia del color.

In [ ]:
!pip install --upgrade --quiet colour-science
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import io
import PIL
import matplotlib.pyplot as plt
import colour

En el bloque que encontramos a continuación, se hace uso de líneas de código en javascript dentro de lo que se ha declara como python. 
Una de las líneas que podemos resaltar es la de "constrait"

var constraints = {video: {facingMode: "environment"}} 

const stream = await navigator.mediaDevices.getUserMedia(constraints); 

pues aquí es donde se especifica el uso de la camara, ya sera la frontal o la trasera. Nosotros hemos declarado por default el uso de la camara trasera para un dispositivo móvil como telefonos inteligentes.


In [ ]:
#funcion para encender camara y tomar foto
def take_photo(filename='photo.jpg', quality=0.8):
  #se hace un llamado a una funcion que trabaja con lineas de javascript
  js = Javascript('''
    //variablesglobales
    var x=320; 
    var y=240;
    async function takePhoto(quality) { 
      const div = document.createElement('div'); 
      window.addEventListener('onmousedown',function(evt){ 
        x=evt.clientX; 
        y=evt.clientY;
      },false);
      const video = document.createElement('video'); 
      video.style.display = 'block' 
      //variable para cambio de camara en dispositivo
      var constraints = {video: {facingMode: "environment"}} 
      const stream = await navigator.mediaDevices.getUserMedia(constraints);   
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
    
      await video.play();

      // Resize the output to fit the video element.
      
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => video.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = 100 //video.videoWidth;
      canvas.height = 100 //video.videoHeight;
      ctx = canvas.getContext('2d');
      ctx.strokeStyle = "red";
      ctx.strokeRect(x-20,y-20,40,40);
      ctx.drawImage(video, x-5, y-5,100,100,0,0,200,200);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


Particularmente en estas lineas de codigo se encuentra la selección del recuadro a analizar.

In [ ]:
const canvas = document.createElement('canvas');
      canvas.width = 100 //video.videoWidth;
      canvas.height = 100 //video.videoHeight;
      ctx = canvas.getContext('2d');
      ctx.strokeStyle = "red";
      ctx.strokeRect(x-20,y-20,40,40);
      ctx.drawImage(video, x-5, y-5,100,100,0,0,200,200);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);

Para el siguiente bloque de código, se requirio que previamente se tuvieran los datos de los colores, y se acomodaron por orden de color RGB y en un plano XYZ, el cual nos sirvio para poder hacer una gráfica de gamut, como la que vimos al inicio de este documento. 

En este caso solamente los anotamos de forma manual para posteriormente usarse sobre el cuadro de la imágen seleccionada.

In [ ]:
import cv2
import numpy as np
#cargamos los datos de xyz y rgb, datos tomados del documento de google sheets
xyz= [[126,89,61],[214,148,108],[110,122,140],[96,117,52],[148,131,157],[122,193,146],[243,136,26],[94,89,155],
[218,93,88],[101,62,98],[170,204,31],[240,182,0],[66,62,135],[79,165,55],[191,68,48],[250,219,0],[206,85,138],
[57,133,147],[255,254,215],[225,210,181],[181,171,147],[136,127,108],[96,90,77],[59,55,47]]

rgb=[[95,	48,	32],[251,	154,	137],[52,	87,	168],[40,	75,	19],[115,	94,	195],[29,	201,	189],[252,	120,	11],[39,	26,	193],[251,	55,	75],
[56,	12,	75],[136,	220,	10],[253,	169,	9],[13,	0,	152],[19,	153,	32],[213,	30,	24],[255,	238,	10],[241,	43,	156],[11,	96,	186],
[254,	254,	254],[233,	232,	238],[175,	173,	186],[43,	41,	52],[40,	39,	47],[11,	10,	15]]
###############################
img = cv2.imread('photo.jpg',cv2.IMREAD_UNCHANGED)

Y por último, el siguiente bloque de código. Realiza las operaciones necesarias para abrir un archivo en Google Sheets y escribir en este los resultados de la medición de color del cuadro de la imágen.

In [ ]:
pip install --upgrade --quiet gspread
from google.colab import auth
auth.authenticate_user()
import time
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

med = [np.median(img[:,:,0]), np.median(img[:,:,1]), np.median(img[:,:,2])] #Calcular el promedio de los píxeles
print("Media", med) #Imprimimos el valor que tiene la media.


correccionColor = colour.colour_correction(med, xyz, rgb, 'Finlayson 2015')
listaCorreccion = list(correccionColor)#Casteamos los datos obtenidos de la correción

print("Correccion", listaCorreccion)#Imprimimos la lista que contiene los datos corregidos.
worksheet= gc.open('Lab2_a').sheet1 #apertura de hoja
cell_list= worksheet.range('A2:C2') #seleccion de rango
cell_list2= worksheet.range('D2:F2')
cell_list3= worksheet.range('H2:H2')

i=0
j=0

#Recorremos la lista para los promedios
for cell in cell_list: #for para llenar con los datos de la media
  cell.value = med[i]
  print(med[i])
  i = i + 1
worksheet.update_cells(cell_list) #Actualizamos la operación

#Forzamos a hacer los valores obtenidos de la listaCorreción flotantes y ademas de redondearlos con la función round.
listaCorrecion = list(correccionColor)
listaCorreccion1 = []
for naux in range(0,3):
  listaCorreccion1.append(float(round(listaCorreccion[naux],4)))
print(listaCorreccion1)

#Recorremos la lista para la correción del color
for cell in cell_list2 : #for para llenar con los datos de la correción de colores
  cell.value = float(listaCorreccion1[j])
  print(float(listaCorreccion1[j]))
  j = j + 1
worksheet.update_cells(cell_list2)#Actualizamos la operación

#Metemos la fecha.
for cell in cell_list3: #for para llenar con los datos de la fecha
  cell.value = time.strftime("%d/%m/%y")
worksheet.update_cells(cell_list3)#Actualizamos la operación

# Evaluación

En las pruebas llegamos a tener los siguientes resultados.
Se tomo la siguiente captura a la imágen
![texto alternativo](https://drive.google.com/uc?id=1ILRIgf54hyGjIc0u-3KqlBl4MfcdMHww)

De la cúal el recuadro a analizar, fue el siguiente

![texto alternativo](https://drive.google.com/uc?id=1xc3c0Gg25ZAKuWCMGyojnH42Ftbz_jpj)

En el analisis logramos obtener los siguientes resultados. Como datos:

![texto alternativo](https://drive.google.com/uc?id=1IgkRFS3q08e6CvM1OfKFiJz-lcw1P6Yh)

Donde los tres primeros datos, son los datos de la media de color en RGB y los siguientes tres son los datos con la corrección de color.
Los cuales son registrados en una tabla dentro de un documento de Google Sheets, donde además se registra la fecha de análisis.

![texto alternativo](https://drive.google.com/uc?id=1V6kb9MjSKuuhfmTy-mQWTK8Hby8YCbAO)

# Conclusiones

Como pudimos valorar, el analizar un aspecto tan sencillo como una imagen tomada por un dispositivo, es algo que la tecnología actual nos permite hacer desde la accesibilidad de nuestros hogares. Por consiguiente la pregunta sería, ¿de que sirve todo esto? bueno, el analizar una imágen nos sirve de mucho en la actualidad, porque como se ha visto analizar un color nos brinda mucha información, por ello el colorimetro es una herramienta muy útil en la quimica por ejemplo. De igual forma corregir el color de una simple imágen nos brinda aun más información, la cual puede ser ocupada para analizar comportamientos y predecir futuros cambios. 


# Referencias

https://es.wikipedia.org/wiki/Color%C3%ADmetro

http://www.metas.com.mx/guiametas/La-Guia-MetAs-09-07-Medicion-de-color.pdf

